In [ ]:
import numpy as np
import math
import time
import random
import itertools
import queue
import pandas as pd
from IPython.display import display, Markdown

# Initialisation

In [ ]:
matrix = np.array([
    [34, 20, 57, 47, 62, 40, 74, 94,  9, 62, 86, 13, 78, 46, 83, 52, 13, 70, 40, 60],
    [ 5, 48, 80, 43, 34,  2, 87, 68, 28, 84, 30, 35, 42, 39, 85, 34, 36,  9, 96, 84],
    [86, 35,  5, 93, 74, 12, 40, 95, 80,  6, 92, 14, 83, 49, 36, 38, 43, 89, 94, 33],
    [28, 39, 55, 21, 25, 88, 59, 40, 90, 18, 33, 10, 59, 92, 15, 77, 31, 85, 85, 99],
    [ 8, 91, 45, 55, 75, 18, 59, 86, 45, 89, 11, 54, 38, 41, 64, 98, 83, 36, 61, 19]
])
m=matrix.shape[0]
print(m)
n=matrix.shape[1]
print(n)
cost = np.transpose(matrix)
Npop=5

5
20


In [ ]:
import numpy as np
#random
def initialization_random(Npop, n):
    pop = []
    for _ in range(Npop):
        p = list(np.random.permutation(n))
        pop.append(p)
    return pop

In [ ]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)

    # Check if the sequence is empty
    if num_jobs == 0:
        # Return a default value (you may choose 0 or another suitable value)
        return 0

    completion_times = np.zeros((num_jobs, num_machines))

    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]

    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]

    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [ ]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    """
    Orders jobs in descending order of their total processing times across all machines.

    Args:
    - processing_times: A 2D numpy array where each row represents a job and each column represents a machine.

    Returns:
    - A list of job indices sorted by their total processing times in descending order.
    """
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

def insertion(sequence, position, value):
    """
    Inserts a given value at a specified position in a sequence.

    """
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq
def neh_algorithm(processing_times):
    """
    Implements the NEH algorithm

    Args:
    - processing_times: A 2D numpy array where each row represents a job and each column represents a machine.

    Returns:
    - The sequence of jobs that minimizes the makespan and the corresponding makespan value.
    """
    # Order jobs by their total processing times in descending order
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)

    # Initialize the sequence with the first two jobs, choosing the order that minimizes the makespan
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]  # Remove the first two jobs from the ordered list

    # Iteratively add the remaining jobs to the sequence in the best position
    for job in ordered_sequence:
        Cmax = float('inf')  # Initialize the minimum makespan found so far
        best_sequence = []  # Initialize the best sequence found so far
        # Try inserting the current job in all possible positions and choose the one with the lowest makespan
        for i in range(len(sequence) + 1):
            new_sequence = insertion(sequence, i, job)  # Generate a new sequence with the current job inserted
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)  # Evaluate the makespan of the new sequence
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval  # Update the minimum makespan if a better one is found
                best_sequence = new_sequence  # Update the best sequence found so far
        sequence = best_sequence


         # Update the sequence to the best one found in this iteration

    # Compute the makespan of the final sequence
    Cmax = evaluate_sequence(sequence, processing_times)
    return sequence, Cmax

In [ ]:
import random

def generate_perturbed_solution(solution):
    """
    Generates a perturbed solution by randomly swapping adjacent jobs.
    """
    perturbed_solution = solution.copy()
    num_jobs = len(perturbed_solution)
    # Perform a random number of swaps (can be adjusted based on problem characteristics)
    num_swaps = random.randint(1, num_jobs // 2)  # Limit the number of swaps to prevent excessive perturbation
    for _ in range(num_swaps):
        # Randomly select two adjacent jobs to swap
        idx = random.randint(0, num_jobs - 2)  # Ensure that idx is within bounds
        perturbed_solution[idx], perturbed_solution[idx + 1] = perturbed_solution[idx + 1], perturbed_solution[idx]
    return perturbed_solution


In [ ]:
def initialization(solution, processing_times, Npop):
    """
    Generates Npop solutions by perturbing the given solution.
    """
    solutions = [(solution, evaluate_sequence(solution, processing_times))]
    for _ in range(Npop - 1):
        perturbed_solution = generate_perturbed_solution(solution)
        makespan = evaluate_sequence(perturbed_solution, processing_times)
        solutions.append((perturbed_solution, makespan))
    # Sort solutions by makespan and keep only the top Npop solutions
    solutions.sort(key=lambda x: x[1])
    return solutions[:Npop]

In [ ]:
import numpy as np

def lpt_heuristic(processing_times):
    # Calculate the total processing time for each job
    total_processing_times = np.sum(processing_times, axis=1)
    # Sort jobs by their total processing time in decreasing order
    sorted_jobs = np.argsort(-total_processing_times)
    return list(sorted_jobs)

# Example usage
processing_times = cost  # Example processing times matrix
solution=lpt_heuristic(processing_times)
solutions = initialization(solution,processing_times,Npop )
population = [solution[0] for solution in solutions]
print(population)
for ind in population:
    makespan = calculateObj(ind)
    print(f'Schedule: {ind}, Makespan: {makespan}')


[[18, 7, 6, 12, 15, 19, 14, 17, 4, 13, 9, 3, 8, 2, 1, 10, 16, 0, 5, 11], [7, 18, 6, 12, 15, 17, 14, 19, 4, 13, 9, 3, 8, 2, 10, 1, 16, 0, 5, 11], [7, 18, 6, 12, 15, 19, 17, 14, 4, 13, 9, 3, 8, 10, 2, 1, 16, 0, 5, 11], [7, 18, 12, 15, 6, 19, 17, 14, 13, 4, 3, 9, 8, 2, 10, 16, 1, 0, 11, 5], [7, 18, 12, 6, 17, 15, 19, 14, 13, 9, 4, 3, 8, 10, 2, 1, 16, 0, 11, 5]]


NameError: name 'calculateObj' is not defined

# Makespane

In [ ]:

def calculateObj(sol):
    qTime = queue.PriorityQueue()

    qMachines = []
    for i in range(m):
        qMachines.append(queue.Queue())

    for i in range(n):
        qMachines[0].put(sol[i])

    busyMachines = []
    for i in range(m):
        busyMachines.append(False)

    time = 0

    job = qMachines[0].get()
    qTime.put((time+cost[job][0], 0, job))
    busyMachines[0] = True

    while True:
        time, mach, job = qTime.get()
        if job == sol[n-1] and mach == m-1:
            break
        busyMachines[mach] = False
        if not qMachines[mach].empty():
                j = qMachines[mach].get()
                qTime.put((time+cost[j][mach], mach, j))
                busyMachines[mach] = True
        if mach < m-1:
            if busyMachines[mach+1] == False:
                qTime.put((time+cost[job][mach+1], mach+1, job))
                busyMachines[mach+1] = True
            else:
                qMachines[mach+1].put(job)
    return time


# ParentSelection

In [ ]:
import numpy as np

def selectionRANK(pop,Npop):
    popObj = []
    for i in range(len(pop)):
        popObj.append([calculateObj(pop[i]), i])

    popObj.sort()  # Sort based on objective value

    distr = []
    distrInd = []

    for i in range(len(pop)):
        distrInd.append(popObj[i][1])
        prob = (2 * (i + 1)) / (len(pop) * (len(pop) + 1))
        distr.append(prob)

    parents = []
    selected_pairs = set()

    while len(parents) < len(pop):
        parent_pair = tuple(np.random.choice(distrInd, 2, p=distr))
        # Ensure the selected pair is unique
        if parent_pair not in selected_pairs:
            selected_pairs.add(parent_pair)
            parents.append(list(parent_pair))

    return parents

# Example usage
pop = population
parents = selectionRANK(pop,Npop)
print(parents)


[[3, 4], [2, 0], [2, 1], [2, 4], [4, 4]]


In [ ]:
import random

def roulette_wheel_selection(population, Npop):
    # Calculate fitness for each individual in the population
    fitnesses = [calculateObj(individual) for individual in population]

    total_fitness = sum(fitnesses)
    selection_probs = [f / total_fitness for f in fitnesses]
    cumulative_probs = [sum(selection_probs[:i+1]) for i in range(len(selection_probs))]

    def select_individual():
        random_number = random.random()
        for individual, cumulative_prob in zip(population, cumulative_probs):
            if random_number < cumulative_prob:
                return individual
        return population[-1]  # Fallback in case of rounding errors

    selected_pairs = set()
    pairs_indexes = []

    while len(pairs_indexes) < Npop:
        parent1_index = population.index(select_individual())
        parent2_index = population.index(select_individual())

        # Ensure a unique pair
        if parent1_index != parent2_index:
            pair = (min(parent1_index, parent2_index), max(parent1_index, parent2_index))
            if pair not in selected_pairs:
                selected_pairs.add(pair)
                pairs_indexes.append(pair)

    return pairs_indexes

# Example usage


# Example usage
population = [solution[0] for solution in solutions]
Npop = len(population)  # Assuming Npop is defined somewhere
print(roulette_wheel_selection(population,Npop))



[(0, 4), (0, 2), (0, 1), (2, 4), (1, 2)]


In [ ]:
import random

def tournament_selection(population, Npop, tournament_size=2):
    fitnesses = [calculateObj(individual) for individual in population]
    selected_pairs = set()
    pairs_indexes = []

    while len(pairs_indexes) < Npop:
        # Randomly select individuals for the tournament
        tournament_individuals = random.sample(range(len(population)), tournament_size)

        # Choose the best individual from the tournament
        best_individual_index = max(tournament_individuals, key=lambda i: fitnesses[i])

        # Randomly select another tournament
        tournament_individuals2 = random.sample(range(len(population)), tournament_size)

        # Choose the best individual from the second tournament
        best_individual_index2 = max(tournament_individuals2, key=lambda i: fitnesses[i])

        # Ensure a unique pair
        if best_individual_index != best_individual_index2:
            pair = (min(best_individual_index, best_individual_index2), max(best_individual_index, best_individual_index2))
            if pair not in selected_pairs:
                selected_pairs.add(pair)
                pairs_indexes.append(pair)

    return pairs_indexes

# Example usage
population = [solution[0] for solution in solutions]
fit = [solution[1] for solution in solutions]
print(roulette_wheel_selection(population,Npop))


[(3, 4), (1, 4), (1, 3), (2, 4), (0, 3)]


# Crossover


In [ ]:
def c1_oper(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:1])
    child = list(parents[0])
    for i in range(pos[0], len(child)):
        child[i] = -1
    p = -1
    for i in range(pos[0], len(child)):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    return child

In [ ]:
def lox_oper(parents):
    pos = list(np.random.permutation(np.arange(n-1)+1)[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    child = list(parents[0])

    for i in range(0, pos[0]):
        child[i] = -1
    for i in range(pos[1],len(child)):
        child[i]= -1
    p = -1
    for i in range(0, pos[0]):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    for i in range(pos[1], len(child)):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break
    return child

In [ ]:
def pox_oper(parents):
    m=int(n/2)
    pos = list(np.random.permutation(np.arange(n-1)+1)[:m])
    child = list(parents[0])
    for i in range(0,len(child)):

        if i not in pos :
            child[i]=-1
    p = -1
    for i in range(0, len(child)):
        if i not in pos :
            while True:
                p = p + 1
                if parents[1][p] not in child:
                    child[i] = parents[1][p]
                    break
    return child

# Mutation

In [ ]:
def swap(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    m=sol[pos[0]]
    sol[pos[0]]=sol[pos[1]]
    sol[pos[1]]=m

    return sol

In [ ]:
def INV(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t
    i=pos[0]
    j=pos[1]
    while (j >= i):
        m=sol[i]
        sol[i]=sol[j]
        sol[j]=m
        i=i+1
        j=j-1

    return sol

In [ ]:
def decalage(sol):
    pos = list(np.random.permutation(np.arange(n))[:2])
    if pos[0] > pos[1]:
        t = pos[0]
        pos[0] = pos[1]
        pos[1] = t

    remJob = sol[pos[1]]

    for i in range(pos[1], pos[0], -1):
        sol[i] = sol[i-1]

    sol[pos[0]] = remJob

    return sol

# Replacement

In [ ]:
def elitistUpdate(oldPop, newPop):
    bestSolInd = 0
    bestSol = calculateObj(oldPop[0])

    for i in range(1, len(oldPop)):
        tempObj = calculateObj(oldPop[i])
        if tempObj < bestSol:
            bestSol = tempObj
            bestSolInd = i

    rndInd = random.randint(0,len(newPop)-1)

    newPop[rndInd] = oldPop[bestSolInd]

    return newPop

In [ ]:
def replace_population(oldPop, newPop):

    # Ensure the old population is completely replaced by the new one
    oldPop[:] = newPop[:]
    return oldPop


In [ ]:
import random

def steady_state_replacement(oldPop, newPop ):
    """
    Performs steady-state replacement by replacing the worst individuals in the old population
    with the best individuals from the new population.

    Args:
        oldPop (list): The old population of individuals.
        newPop (list): The new population of individuals.
        calculateObj (function): The objective function to evaluate fitness.
        num_replacements (int): Number of individuals to replace in the old population.

    Returns:
        list: The updated population after replacement.
    """
    num_replacements = random.randint(1, pop_size // 2)
    # Evaluate fitness of the old population
    oldPop_fitness = [(individual, calculateObj(individual)) for individual in oldPop]
    oldPop_fitness.sort(key=lambda x: x[1])  # Sort by fitness (ascending)

    # Evaluate fitness of the new population
    newPop_fitness = [(individual, calculateObj(individual)) for individual in newPop]
    newPop_fitness.sort(key=lambda x: x[1], reverse=True)  # Sort by fitness (descending)

    # Replace the worst individuals in the old population with the best from the new population
    for i in range(num_replacements):
        oldPop_fitness[-(i+1)] = newPop_fitness[i]

    # Extract the individuals from the (individual, fitness) tuples
    updated_population = [individual for individual, fitness in oldPop_fitness]

    return updated_population



In [ ]:
def findBestSolution(pop):
    bestObj = calculateObj(pop[0])
    avgObj = bestObj
    bestInd = 0
    for i in range(1, len(pop)):
        tObj = calculateObj(pop[i])
        avgObj = avgObj + tObj
        if tObj < bestObj:
            bestObj = tObj
            bestInd = i

    return bestInd, bestObj, avgObj/len(pop)

In [ ]:
import random
import time

def genetic_algorithm(method, selection, crossover, mutation, population_update, cost, Npop, stopGeneration, Pc, Pm):
    # Initialize population
    if method == initialization_random:
        population = method(Npop,n)
    else:
        solution = method(cost)
        population = initialization(solution[0], cost, Npop)
        population = [solution[0] for solution in population]

    # Evaluate initial population
    objectives = [evaluate_sequence(individual, cost) for individual in population]

    for i in range(stopGeneration):
        # Select parents
        parents = selection(population,Npop)

        # Apply crossover
        childs = [crossover([population[p[0]], population[p[1]]]) for p in parents if random.random() < Pc]

        # Apply mutation
        for child in childs:
            if random.random() < Pm:
                child = mutation(child)

        # Update population
        population = population_update(population, childs)

        # Evaluate the new population
        objectives = [evaluate_sequence(individual, cost) for individual in population]

    # Find the best solution
    bestSol, bestObj, avgObj = findBestSolution(population)

    return population, bestSol, bestObj, avgObj

# Example usage:

# Set parameters
Npop = 20
stopGeneration = 1000
Pc = 1
Pm = 1

# Run the algorithm
t1 = time.perf_counter()

population, bestSol, bestObj, avgObj = genetic_algorithm(
    method=initialization_random,
    selection=roulette_wheel_selection,
    crossover=lox_oper,
    mutation=decalage,
    population_update=elitistUpdate,
    cost=cost,
    Npop=Npop,
    stopGeneration=stopGeneration,
    Pc=Pc,
    Pm=Pm
)

# Stop Timer
t2 = time.perf_counter()

# Results Time
print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)


Population:
[[11, 4, 5, 15, 12, 1, 17, 16, 6, 8, 2, 13, 10, 9, 14, 3, 7, 18, 19, 0], [1, 6, 0, 3, 15, 9, 12, 4, 8, 2, 14, 7, 19, 17, 18, 5, 11, 10, 16, 13], [10, 11, 3, 8, 13, 6, 2, 16, 9, 5, 19, 0, 7, 15, 18, 14, 4, 17, 12, 1], [0, 3, 13, 2, 18, 15, 8, 9, 7, 10, 11, 4, 6, 17, 1, 14, 19, 5, 12, 16], [3, 13, 2, 15, 8, 7, 10, 11, 4, 16, 14, 1, 17, 9, 5, 18, 19, 0, 12, 6], [7, 15, 12, 16, 11, 1, 6, 8, 2, 13, 10, 0, 19, 18, 14, 4, 3, 17, 5, 9], [1, 6, 0, 12, 9, 15, 8, 10, 11, 4, 16, 14, 17, 5, 19, 7, 3, 18, 13, 2], [1, 10, 12, 11, 14, 15, 9, 4, 6, 3, 2, 13, 17, 0, 19, 5, 18, 16, 8, 7], [1, 12, 15, 8, 17, 16, 2, 14, 11, 9, 4, 10, 5, 6, 7, 18, 13, 0, 19, 3], [10, 4, 6, 11, 7, 13, 8, 2, 3, 15, 16, 14, 17, 9, 5, 18, 19, 0, 12, 1], [16, 4, 5, 15, 12, 17, 11, 1, 6, 8, 2, 13, 10, 9, 14, 3, 7, 18, 19, 0], [3, 13, 2, 7, 16, 11, 1, 6, 18, 15, 8, 9, 17, 10, 4, 19, 14, 5, 12, 0], [3, 13, 11, 18, 15, 1, 9, 17, 7, 10, 8, 2, 4, 6, 19, 14, 5, 12, 0, 16], [10, 4, 3, 13, 0, 8, 7, 9, 17, 15, 2, 16, 14, 5, 18

# Haut Niveau

In [ ]:
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the FSSP problem
class FSSP:
    def __init__(self, num_jobs, num_machines, processing_times):
        self.num_jobs = num_jobs
        self.num_machines = num_machines
        self.processing_times = processing_times

    def calculate_makespan(self, schedule):
        # Calculate the makespan given a schedule
        completion_times = np.zeros((self.num_jobs, self.num_machines))
        for job in schedule:
            for machine in range(1, self.num_machines):
                completion_times[job][machine] = max(completion_times[job][machine - 1], completion_times[job - 1][machine]) + self.processing_times[job][machine]
            completion_times[job][0] = completion_times[job - 1][0] + self.processing_times[job][0]
        return np.max(completion_times)

# Define the RL agent using a neural network
class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.model = self._build_model()
        self.memory = []
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995

    def _build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def learn(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, rewards, next_state, done in minibatch:
            target = rewards
            if not done:
                target = rewards + self.gamma * np.amax(self.model.predict(next_state), axis=1)
            target_f = self.model.predict(state)
            target_f[np.arange(len(target)), action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Define the niching strategy
class NichingStrategy:
    def __init__(self, population_size, num_subpopulations):
        self.population_size = population_size
        self.num_subpopulations = num_subpopulations
        self.subpopulation_size = population_size // num_subpopulations
        # Initialize niching parameters

    def maintain_diversity(self, population):
        # Apply niching strategy to maintain population diversity
        pass

# Define genetic operators
def initialize_population(fssp, method):
    # Initialize population using NEH or LST
    if method == 'NEH':
        pass  # Placeholder for NEH initialization
    elif method == 'LST':
        pass  # Placeholder for LST initialization

def mutate(schedule, method):
    # Apply mutation (swap or inverse) to the schedule
    pass

def select(population, method):
    # Select individuals from the population using ranking or tournament
    pass

def replace(population, new_individuals, method):
    # Replace individuals in the population using elitism or steady strategy
    pass

def crossover(parent1, parent2, method):
    # Perform crossover (e.g., COP - Cyclic Order Crossover)
    pass

# Main loop
def main():
    # Initialize FSSP, RL agents, and niching strategy
    num_jobs = 5
    num_machines = 3
    processing_times = np.random.randint(1, 10, size=(num_jobs, num_machines))
    fssp = FSSP(num_jobs, num_machines, processing_times)
    num_subpopulations = 2
    population_size = 10
    state_size = num_jobs * num_machines  # State size = flattened processing times matrix
    action_size = 2  # Number of genetic operators: mutate, crossover
    agents = [RLAgent(state_size, action_size) for _ in range(num_subpopulations)]
    niching = NichingStrategy(population_size, num_subpopulations)
    population = [[] for _ in range(num_subpopulations)]

    # Initialize population with NEH or LST
    for i in range(num_subpopulations):
        population[i] = initialize_population(fssp, method='NEH')  # or 'LST'

    # Main loop
    num_episodes = 10
    for episode in range(num_episodes):
        for i, agent in enumerate(agents):
            subpopulation = population[i]
            # Agent selects an action (genetic operator)
            state = np.reshape(processing_times, (1, state_size))  # Flatten processing times matrix as state
            action = agent.act(state)

            # Apply the selected genetic operator
            new_schedules = []
            if action == 0:  # Mutate
                new_schedules = [mutate(s, method='swap') for s in subpopulation]
            elif action == 1:  # Crossover
                parents = select(subpopulation, method='tournament')
                new_schedules = [crossover(parents[0], parents[1], method='COP')]
            # Placeholder for other operators

            # Update the population and learn
            rewards = [fssp.calculate_makespan(s) for s in new_schedules]
            next_state = np.reshape(processing_times, (1, state_size))  # Flatten processing times matrix as next state
            agent.memory.append((state, action, rewards, next_state, done))
            agent.learn(batch_size=32)
            population[i] = replace(subpopulation, new_schedules, method='elitism')

        # Apply niching strategy to maintain diversity
        niching.maintain_diversity(population)

        # Decay exploration rate
        for agent in agents:
            agent.epsilon *= agent.epsilon_decay

    # After training, select the best solutions from each subpopulation
    best_solutions = [min(subpop, key=lambda s: fssp.calculate_makespan(s)) for subpop in population]
    print("Best solutions:", best_solutions)

if __name__ == "__main__":
    main()


TypeError: 'NoneType' object is not iterable

In [ ]:
import random
import numpy as np
import time

class RLHyperHeuristic:
    def __init__(self, low_level_heuristics, calculateObj, initial_temp=1.0, cooling_rate=0.99):
        self.low_level_heuristics = low_level_heuristics
        self.calculateObj = calculateObj
        self.q_table = np.zeros(len(low_level_heuristics))
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.selected_operations = []

    def select_heuristic(self, temperature):
        if random.random() < temperature:
            selected_index = random.choice(range(len(self.low_level_heuristics)))
        else:
            selected_index = np.argmax(self.q_table)
        self.selected_operations.append(selected_index)
        return selected_index

    def update_q_table(self, heuristic_index, reward):
        self.q_table[heuristic_index] += reward

    def apply_heuristics(self, population, cost, Npop, stopGeneration):
        temperature = self.initial_temp
        for i in range(stopGeneration):
            heuristic_index = self.select_heuristic(temperature)
            selected_heuristic = self.low_level_heuristics[heuristic_index]

            # Apply selected heuristic to the population
            new_population = selected_heuristic(population, cost, Npop)
            new_objectives = [self.calculateObj(individual) for individual in new_population]

            # Calculate reward as improvement in objective value
            old_objectives = [self.calculateObj(individual) for individual in population]
            reward = np.mean(old_objectives) - np.mean(new_objectives)

            self.update_q_table(heuristic_index, reward)

            # Update population and decrease temperature
            population = new_population
            temperature *= self.cooling_rate

        return population

    def print_selected_operations(self, operation_names):
        print("Selected Operations in Each Phase:")
        for index in self.selected_operations:
            print(operation_names[index])



In [ ]:
import random
import time

def genetic_algorithm(method, selection, crossover, mutation, population_update, cost, Npop, stopGeneration, Pc, Pm):
    # Initialize population
    if method == initialization_random:
        population = method(Npop, len(cost))
    else:
        solution = method(cost)
        population = initialization(solution[0], cost, Npop)
        population = [solution[0] for solution in population]

    # Evaluate initial population
    objectives = [evaluate_sequence(individual, cost) for individual in population]

    for i in range(stopGeneration):
        # Select parents
        parents = selection(population, Npop)

        # Apply crossover
        childs = [crossover([population[p[0]], population[p[1]]]) for p in parents if random.random() < Pc]

        # Apply mutation
        for child in childs:
            if random.random() < Pm:
                child = mutation(child)

        # Update population
        population = population_update(population, childs)

        # Evaluate the new population
        objectives = [evaluate_sequence(individual, cost) for individual in population]

    # Find the best solution
    bestSol, bestObj, avgObj = findBestSolution(population)

    return population, bestSol, bestObj, avgObj

# Example usage:

# Define low-level heuristics lists
initialization_methods = [initialization_random, neh_algorithm, lpt_heuristic]
selection_methods = [selectionRANK, roulette_wheel_selection, tournament_selection]
crossover_methods = [c1_oper, lox_oper, pox_oper]
mutation_methods = [swap, INV, decalage]
population_update_methods = [elitistUpdate, replace_population, steady_state_replacement]

# Initialize RL HyperHeuristic with all low-level methods
rl_hyper_heuristic = RLHyperHeuristic(
    low_level_heuristics=[
        initialization_methods,
        selection_methods,
        crossover_methods,
        mutation_methods,
        population_update_methods
    ],
    calculateObj=calculateObj
)

# Randomly generate parameters
Npop = random.randint(5, 30)
stopGeneration = random.randint(700, 1500)
Pc = random.uniform(0.5, 1)
Pm = random.uniform(0.5, 1)

# Run the algorithm
t1 = time.perf_counter()

population, bestSol, bestObj, avgObj = genetic_algorithm(
    method=random.choice(initialization_methods),
    selection=random.choice(selection_methods),
    crossover=random.choice(crossover_methods),
    mutation=random.choice(mutation_methods),
    population_update=random.choice(population_update_methods),
    cost=cost,
    Npop=Npop,
    stopGeneration=stopGeneration,
    Pc=Pc,
    Pm=Pm
)

# Stop Timer
t2 = time.perf_counter()

# Results Time
print("Population:")
print(population)
print()

print("Solution:")
print(population[bestSol])
print()

print("Objective Value:")
print(bestObj)
print()

print("Average Objective Value of Population:")
print("%.2f" % avgObj)
print()

print("CPU Time (s)")
timePassed = (t2 - t1)
print("%.2f" % timePassed)


In [ ]:
import random
import numpy as np

class RLHyperHeuristic:
    def __init__(self, low_level_heuristics, calculateObj, initial_temp=1.0, cooling_rate=0.99):
        self.low_level_heuristics = low_level_heuristics
        self.calculateObj = calculateObj
        self.q_table = np.zeros((len(low_level_heuristics), len(low_level_heuristics[0])))  # Q-values for each phase
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.selected_operations = []

    def select_heuristics(self, temperature):
        selected_indices = []
        for phase in range(len(self.low_level_heuristics)):
            if random.random() < temperature:
                selected_index = random.choice(range(len(self.low_level_heuristics[phase])))
            else:
                selected_index = np.argmax(self.q_table[phase])
            selected_indices.append(selected_index)
            self.selected_operations.append(selected_index)
        return selected_indices

    def update_q_table(self, heuristic_indices, reward):
        for phase, index in enumerate(heuristic_indices):
            self.q_table[phase, index] += reward

    def print_selected_operations(self, operation_names):
        print("Selected Operations in Each Phase:")
        for phase, index in enumerate(self.selected_operations):
            print(f"Phase {phase + 1}: {operation_names[phase][index]}")

# Define low-level heuristics lists
initialization_methods = [initialization_random, neh_algorithm, lpt_heuristic]
selection_methods = [selectionRANK, roulette_wheel_selection, tournament_selection]
crossover_methods = [c1_oper, lox_oper, pox_oper]
mutation_methods = [swap, INV, decalage]
population_update_methods = [elitistUpdate, replace_population, steady_state_replacement]

# Define the names of the low-level heuristics
operation_names = [
    ["initialization_random", "neh_algorithm", "lpt_heuristic"],
    ["selectionRANK", "roulette_wheel_selection", "tournament_selection"],
    ["c1_oper", "lox_oper", "pox_oper"],
    ["swap", "INV", "decalage"],
    ["elitistUpdate", "replace_population", "steady_state_replacement"]
]

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

class RLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(0, self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def learn(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
        target_f = self.model.predict(state)
        target_f[0][action] = target
        self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
def genetic_algorithm_with_RL(cost, Npop, stopGeneration, Pc, Pm):
    state_size = 1  # Simplified state representation
    action_size = 3  # Three actions: 0 for crossover, 1 for mutation, 2 for selection
    agent = RLAgent(state_size, action_size)

    population = initialization_random(Npop, n)
    objectives = [evaluate_sequence(individual, cost) for individual in population]

    for generation in range(stopGeneration):
        state = np.array([[np.mean(objectives)]])  # State could be the average fitness
        action = agent.act(state)

        if action == 0:
            parents = roulette_wheel_selection(population, Npop)
            children = [lox_oper([population[p[0]], population[p[1]]]) for p in parents if np.random.rand() < Pc]
        elif action == 1:
            children = [decalage(ind) if np.random.rand() < Pm else ind for ind in population]
        else:
            children = population  # No operation

        new_population = elitistUpdate(population, children)
        new_objectives = [evaluate_sequence(individual, cost) for individual in new_population]

        next_state = np.array([[np.mean(new_objectives)]])
        reward = -np.mean(new_objectives)  # Negative because lower makespan is better
        done = generation == stopGeneration - 1
        agent.learn(state, action, reward, next_state, done)

        population, objectives = new_population, new_objectives

    best_index, best_obj, avg_obj = findBestSolution(population)
    return population, best_index, best_obj, avg_obj

In [ ]:
# Parameters
Npop = 20
stopGeneration = 10
Pc = 0.8
Pm = 0.2

# Run the enhanced GA
population, best_index, best_obj, avg_obj = genetic_algorithm_with_RL(cost, Npop, stopGeneration, Pc, Pm)
print("Best Solution:", population[best_index])
print("Best Objective:", best_obj)

1/1 [==============================] - 0s 21ms/step
Best Solution: [13, 2, 9, 0, 15, 8, 6, 4, 10, 1, 14, 7, 19, 17, 16, 18, 11, 12, 5, 3]
Best Objective: 1428
